In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from atomphys import Atom, Laser, Transition, State

import matplotlib.pyplot as plt
from math import pi, atan

In [3]:
from atomphys.data.nist import fetch_states

# fetch_states('ca ii', refresh_cache=True)

In [4]:
ca = Atom("Ca+")

In [5]:
ca.states[0].states

4 States (
State(3D3/2, 2D3/2: 0.1244 Ry)
State(3D5/2, 2D5/2: 0.1249 Ry)
State(4P1/2, 2P1/2: 0.2296 Ry)
State(4P3/2, 2P3/2: 0.2316 Ry))

In [6]:
u = ca.units
u.define('planck_terahertz = planck_constant * THz = hTHz')

In [7]:
u.setup_matplotlib()

In [8]:
u.planck_constant

<Unit('planck_constant')>

In [12]:
tt = ca.transitions[0]
tt.wavelength

164786.0850962732 <Unit('nanometer')>

In [11]:
tt

Transition(3D3/2, 2D3/2 <--> 3D5/2, 2D5/2 : λ=1.6479×10⁵ nm, Γ=2π×384 nHz)

In [10]:
tr_729 = ca.transitions(729. * u.nm)

In [ ]:
s_d52 = tr_729.state_f

In [ ]:
s_d52.quantum_numbers

In [ ]:
yb = Atom('Yb+')

In [ ]:
def tr_wavelength(state_i, state_f):
    de = state_f.energy - state_i.energy
    freq = de / u.planck_constant
    wl = (u.c / freq).to('nm') if freq > 0 else None
    return wl

def find_transition_candidate(atom, wavelength, precision=None, state_i=None):
    tr_energy = u.planck_constant * u.c / wavelength
    states = atom.states if state_i is None else [atom.states(state_i)]
    states_f = []
    for state_i in states:
        state_f = atom.states(state_i.energy + tr_energy)
        wl = tr_wavelength(state_i, state_f)
        if wl is not None:
            if precision is not None and abs(wavelength - wl) > precision:
                continue
            print(state_i, state_f, wl)
            states_f.append(state_f)
    return states_f
        
f1 = find_transition_candidate(yb, 638 * u.nm, state_i='2F7/2')[0]
print('---')
f2 = find_transition_candidate(yb, 935 * u.nm, state_i='2D3/2')[0]
print('---')
find_transition_candidate(yb, 935 * u.nm, precision=0.3 * u.nm);
print('---')

In [ ]:
_t = Transition(state_i=yb('2F7/2'), state_f=f1, atom=yb);  #, Gamma = 1 / yb('[3/2]1/2').lifetime)
yb.transitions.append(_t)

_t = Transition(state_i=yb('2D3/2'), state_f=f2, atom=yb);  #, Gamma = 1 / yb('[3/2]1/2').lifetime)
yb.transitions.append(_t)


In [ ]:
yb.transitions

In [ ]:
be = Atom('Be+')

In [ ]:
ba = Atom('Ba+')

In [ ]:
from matplotlib import patches as mpatches

import matplotlib.colors


def _wavelength_to_rgb(wavelength, gamma=0.8):
    ''' taken from http://www.noah.org/wiki/Wavelength_to_RGB_in_Python
    This converts a given wavelength of light to an 
    approximate RGB color value. The wavelength must be given
    in nanometers in the range from 380 nm through 750 nm
    (789 THz through 400 THz).

    Based on code by Dan Bruton
    http://www.physics.sfasu.edu/astro/color/spectra.html
    Additionally alpha value set to 0.5 outside range
    
    https://stackoverflow.com/a/44960748
    '''
    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 750:
        A = 1.
    else:
        A=0.5
    if wavelength < 380:
        wavelength = 380.
    if wavelength >750:
        wavelength = 750.
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0
    return (R,G,B,A)


def _get_state_anchor(state, energy_units):
    e = state.energy.to(energy_units).m
    qn = state.quantum_numbers
    L = qn.get('L', qn.get('K'))
#     L = 1 if L == 0 else L
    e = 1 if e == 0 else e
    anchor = (L, e)
    return anchor
    

def plot_state(ax, state: State, energy_units='hTHz'):
    name = state.name
    anchor = _get_state_anchor(state, energy_units)
    width, height = 0.9, 0
    xy = (anchor[0] - width / 2, anchor[1])
    patch = mpatches.Rectangle(xy, width, height, ec='k', fc='none')
    ax.add_patch(patch)
    ax.relim()
    ax.autoscale_view()
    ax.text(anchor[0], anchor[1] + 0.005, name, ha='center', va='bottom')
    
def plot_transition(ax, tr: Transition, energy_units='hTHz'):
    si, sf = tr.state_i, tr.state_f
    wl = (u.c / tr.energy * u.planck_constant).to('nm')
    color = _wavelength_to_rgb(wl.m)
    ai = _get_state_anchor(si, energy_units)
    af = _get_state_anchor(sf, energy_units)
    dx = af[0] - ai[0]
    dy = af[1] - ai[1]
    midpoint = ai[0] + dx/2, ai[1] + dy/2
    angle = atan(dy / dx) * 180 / pi if dx != 0 else 90
    arrow = mpatches.Arrow(*ai, dx, dy, color=color, width=0.1)
    th2 = ax.text(*midpoint, wl, ha='center', va='center',
            rotation=angle, rotation_mode='anchor',
            transform_rotates_text=True)
    ax.add_patch(arrow)
    ax.relim()
    ax.autoscale_view()

    
atom = yb
states = [s for s in atom.states if s.energy < 1000 * u('hTHz')]

fig, ax = plt.subplots()
for s in states:
    plot_state(ax, s)
    
for tr in atom.states[0].transitions + atom.states('D5/2').transitions[1:2]:
    plot_transition(ax, tr)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4), sharex=True, sharey=True)

def plot_atom(ax, atom):
    states = [s for s in atom.states if s.energy < 1000 * u('hTHz')]
    for s in states:
        plot_state(ax, s)
    
    transitions = [tr for tr in atom.states[0].transitions if tr.energy < 1000 * u('hTHz')]    
    for tr in transitions:
        plot_transition(ax, tr)
    
    
for ax, atom in zip(axes, [ca, yb, ba]):
    plot_atom(ax, atom)
    ax.set_title(atom.name)
ax.set_yscale('log')

In [ ]:
yb.transitions(u.planck_constant * u.c / (935 * u.nm))

In [ ]:
yb.states(0.19 * u.E_h)

In [ ]:
yb.transitions(935.)

In [ ]:
yb('[3/2]1/2').transitions[1].matrix_element

In [ ]:
gamma = 1 / s.lifetime

In [ ]:
[print(s.name, tr_wavelength(yb('2D3/2'), s)) for s in yb.states if '[3/2]' in s.name];

In [ ]:
yb('3[3/2]1/2')

In [ ]:
from atomphys import Transition

In [ ]:
((yb('3[3/2]5/2').energy / u.planck_constant) + (u.c / (935*u.nm))).to('THz')

In [ ]:
tr_wavelength(yb('2D3/2'), yb('3[3/2]5/2'))

In [ ]:
tr_wavelength(yb.states[0], yb('2D3/2'))

In [ ]:
[print(s) for s in yb.states[:10]];

In [ ]:
u('speed_of_light').to_base_units()

In [ ]:
(atom.states[1].energy / u.dirac_constant).to('THz')

In [ ]:
(atom.states[1].energy / (2*pi*u.dirac_constant)).to("THz")

In [ ]:
(u.c / (300 * u.nm)).to('THz')

In [ ]:
s0.up

In [ ]:
s1 = yb.states[0]

In [ ]:
s1.quantum_numbers

In [ ]:
qn = state.quantum_numbers

In [ ]:
qn.get('L', qn.get('K'))

In [ ]:
yb.states[0].quantum_numbers

In [ ]:
for s in yb.states[:10]:
    print(s.name, s.quantum_numbers)